In [11]:
# Append web folder to system path
import sys
sys.path.append('../web')

In [12]:
from db_config import Base
from Database import Station, Availability, Weather
from sqlalchemy import create_engine, func, Column, String, Integer, Double, Boolean
from sqlalchemy.orm import sessionmaker
import json
import sys

In [35]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

In [14]:
# Get the db_info
with open('../dbinfo.json') as f:
    db_info = json.load(f)


USER = db_info['dbConnection']['USER']
PASSWORD = db_info['dbConnection']['PASSWORD']
URI = db_info['dbConnection']['URI']
PORT = db_info['dbConnection']['PORT']
DB = db_info['dbConnection']['DB']

# Create a new session
engine = create_engine(
    'mysql+pymysql://{}:{}@localhost:{}/{}'.format(USER, PASSWORD, PORT, DB), echo=True)
Base.metadata.create_all(bind=engine)
Session = sessionmaker(bind=engine)
session = Session()
print("connected")


2024-03-29 13:37:11,359 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2024-03-29 13:37:11,359 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-03-29 13:37:11,390 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2024-03-29 13:37:11,391 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-03-29 13:37:11,407 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2024-03-29 13:37:11,407 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-03-29 13:37:11,438 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-29 13:37:11,439 INFO sqlalchemy.engine.Engine DESCRIBE `db_on_your_bike`.`stations`
2024-03-29 13:37:11,440 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-03-29 13:37:11,475 INFO sqlalchemy.engine.Engine DESCRIBE `db_on_your_bike`.`availability`
2024-03-29 13:37:11,515 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-03-29 13:37:11,535 INFO sqlalchemy.engine.Engine DESCRIBE `db_on_your_bike`.`weather`
2024-03-29 13:37:11,537 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-03-29 13:37:1

In [15]:
# bike_stations = session.query(Station).all()
# bike_availability = session.query(Availability).filter(
#     Availability.time_updated == func.max(Availability.time_updated).select()).all()
weather = session.query(Weather).all()

# for row in bike_availability:
#     data["stations"][row.station_id] = {
#         "available_bikes": row.available_bikes,
#         "available_bike_stands": row.available_bike_stands
#     }

# for row in bike_stations:
#     data["stations"][row.station_id]["name"] = row.name
#     data["stations"][row.station_id]["latitude"] = row.latitude
#     data["stations"][row.station_id]["longitude"] = row.longitude
# data["weather"] = {
#     "type": weather.type,
#     "temperature": weather.temperature,
#     "humidity": weather.humidity,
#     "wind speed": weather.wind_speed

# }

for row in weather:
    print(row.time_updated, row.type, row.temperature, row.humidity)


2024-03-29 13:37:11,598 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-29 13:37:11,602 INFO sqlalchemy.engine.Engine SELECT weather.time_updated AS weather_time_updated, weather.type AS weather_type, weather.description AS weather_description, weather.temperature AS weather_temperature, weather.feels_like AS weather_feels_like, weather.min_temp AS weather_min_temp, weather.max_temp AS weather_max_temp, weather.humidity AS weather_humidity, weather.wind_speed AS weather_wind_speed, weather.visibility AS weather_visibility, weather.clouds AS weather_clouds, weather.sunrise AS weather_sunrise, weather.sunset AS weather_sunset 
FROM weather
2024-03-29 13:37:11,603 INFO sqlalchemy.engine.Engine [generated in 0.00138s] {}
2024-02-20 16:05:05 Clouds 11.91 81
2024-02-20 16:10:05 Clouds 11.96 80
2024-02-20 16:15:05 Clouds 11.83 81
2024-02-20 16:20:05 Clouds 11.96 80
2024-02-20 16:25:05 Clouds 11.83 81
2024-02-20 16:30:06 Clouds 11.67 79
2024-02-20 16:35:04 Clouds 11.67 79
2024-02-20 16:

In [16]:
bike_availability = session.query(Availability).all()


2024-03-29 13:37:12,561 INFO sqlalchemy.engine.Engine SELECT availability.station_id AS availability_station_id, availability.time_updated AS availability_time_updated, availability.bike_stands AS availability_bike_stands, availability.available_bikes AS availability_available_bikes, availability.available_bike_stands AS availability_available_bike_stands, availability.status AS availability_status 
FROM availability
2024-03-29 13:37:12,565 INFO sqlalchemy.engine.Engine [generated in 0.00442s] {}


In [17]:
availability_df = pd.DataFrame([row.__dict__ for row in bike_availability])


In [19]:
availability_df.drop('_sa_instance_state', axis=1, inplace=True)
availability_df.to_csv('availability.csv')

In [27]:
availability_df.sort_values('time_updated', inplace=True)

In [28]:
weather_df = pd.DataFrame([row.__dict__ for row in weather])
weather_df.drop('_sa_instance_state', axis=1, inplace=True)
weather_df.sort_values('time_updated', inplace=True)
weather_df.head()

,time_updated,temperature,description,min_temp,humidity,visibility,sunrise,type,feels_like,max_temp,wind_speed,clouds,sunset
0,2024-02-20 16:05:05,11.91,few clouds,11.53,81,10000,1708414464,Clouds,11.27,12.36,9.77,20,1708451020
1,2024-02-20 16:10:05,11.96,few clouds,11.53,80,10000,1708414464,Clouds,11.30,12.36,9.77,20,1708451020
2,2024-02-20 16:15:05,11.83,few clouds,11.05,81,10000,1708414464,Clouds,11.18,12.36,9.77,20,1708451020
3,2024-02-20 16:20:05,11.96,few clouds,11.53,80,10000,1708414464,Clouds,11.30,12.36,9.77,20,1708451020
4,2024-02-20 16:25:05,11.83,few clouds,11.05,81,10000,1708414464,Clouds,11.18,12.36,9.77,20,1708451020


In [25]:
weather_df.to_csv('weather.csv')

In [29]:
df = pd.merge_asof(availability_df, weather_df, on='time_updated')

In [32]:
df = df[['station_id', 'time_updated', 'available_bikes', 'available_bike_stands', 'temperature', 'wind_speed', 'humidity']]
df.head()

,station_id,time_updated,available_bikes,available_bike_stands,temperature,wind_speed,humidity
0,1,2024-02-20 16:05:06,2,29,11.91,9.77,81
1,96,2024-02-20 16:05:06,14,16,11.91,9.77,81
2,13,2024-02-20 16:05:06,14,16,11.91,9.77,81
3,95,2024-02-20 16:05:06,3,37,11.91,9.77,81
4,94,2024-02-20 16:05:06,0,40,11.91,9.77,81


In [33]:
df.to_csv('weather_availability_joined.csv')

In [39]:
# One hot encode day of the week
df['weekday'] = df['time_updated'].dt.day_name()
df['hour'] = df['time_updated'].dt.hour
one_hot_encoded = pd.get_dummies(df['weekday'])
df = pd.concat([df, one_hot_encoded], axis=1)
df.drop('time_updated', axis = 1, inplace=True)
df.head()

KeyError: "['time_updated'] not found in axis"